In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
import mapping
from tensorflow.keras.preprocessing import image
import splitfolders
import matplotlib.pyplot as plt
from tensorflow.keras.utils import register_keras_serializable
from custom_layers.MinPooling import MinPooling2D

In [2]:
label_map_marital_status={
    "single_m": "أعزب",
    "single_f": "أنسة",
    "married_m": "متزوج",
    "married_f": "متزوجة",
    "widow": "أرملة",
    "widower": "أرمل",
}

In [8]:
img_size = (64, 64)
batch_size = 32

Marital_status_train_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\Marital status\train",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

Marital_status_val_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\Marital status\val",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

Marital_status_test_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\Marital status\test",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)


normalization_layer = tf.keras.layers.Rescaling(1./255)

class_names_Marital_status = Marital_status_train_ds.class_names
print("Classes:", class_names_Marital_status)
num_classes = len(class_names_Marital_status)

Marital_status_train_ds = Marital_status_train_ds.map(lambda x, y: (normalization_layer(x), y))
Marital_status_val_ds = Marital_status_val_ds.map(lambda x, y: (normalization_layer(x), y))
Marital_status_test_ds = Marital_status_test_ds.map(lambda x, y: (normalization_layer(x), y))

Found 2895 files belonging to 6 classes.
Found 783 files belonging to 6 classes.
Found 459 files belonging to 6 classes.
Classes: ['married_f', 'married_m', 'single_f', 'single_m', 'widow', 'widower']


In [10]:
model = models.Sequential([
    
    layers.Input(shape=(64, 64, 3)),

    layers.Conv2D(16, 3, activation='relu', padding='same'),
    layers.Conv2D(16, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation='relu',padding='same'),
    layers.Conv2D(64, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(128, 3, activation='relu',padding='same'),
    layers.Conv2D(128, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
history = model.fit(Marital_status_train_ds, epochs=10, 
                    validation_data=Marital_status_val_ds)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - accuracy: 0.4359 - loss: 1.3344 - val_accuracy: 0.9847 - val_loss: 0.0534
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 125ms/step - accuracy: 0.9827 - loss: 0.0620 - val_accuracy: 0.9821 - val_loss: 0.0820
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 125ms/step - accuracy: 0.9979 - loss: 0.0087 - val_accuracy: 0.9987 - val_loss: 0.0111
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 20s 116ms/step - accuracy: 0.9990 - loss: 0.0034 - val_accuracy: 0.9974 - val_loss: 0.0046
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 124ms/step - accuracy: 0.9955 - loss: 0.0177 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 126ms/step - accuracy: 1.0000 - loss: 1.1740e-04 - val_accuracy: 1.0000 - val_loss: 1.1632e-04
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 124ms/step - accuracy: 1.0000 - loss: 3.9539e-05 - val_accuracy: 1.0000 - val_loss: 8.1432e-05
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 124ms/step - accuracy: 1.0000 - loss: 2

In [13]:
test_loss, test_acc = model.evaluate(Marital_status_test_ds)

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 1.0000 - loss: 2.0508e-05


In [14]:
model.save("../models/cnn_model_marital_status_2.keras")